<a href="https://colab.research.google.com/github/Jaguar838/llm-zoomcamp/blob/main/HW/workshops/LLM_zoomcamp_RAG_demo/LLM_zoomcamp_RAG_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro dlt -> LanceDB loading example

https://lu.ma/cnpdoc5n

Если вы хотите играть с этим блокнотом и вносить правки в будущем, мы настоятельно рекомендуем сделать копию, поскольку ссылка доступна только для просмотра! Также убедитесь, что вы вошли в свой аккаунт Google, чтобы иметь возможность добавлять секреты.


Прежде чем перейти к более сложному примеру, мы рассмотрим простой пример загрузки данных курса "Вопросы и ответы" в LanceDB.

## Install requirements

Чтобы создать конвейер json -> lancedb, нам нужно установить:
1. dlt с дополнительными возможностями lancedb
2. sentence-transformers: нам нужно использовать модель встраивания для векторизации и хранения данных в LanceDB. Для этого мы выбираем модель с открытым исходным кодом "sentence-transformers/all-MiniLM-L6-v2".

In [ ]:
%%capture
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers

## Load the data

Сначала мы загрузим данные просто в LanceDB, без встраивания. LanceDB хранит и данные, и вставки, а также может встраивать данные и запросы на лету.

Некоторые определения:
* dlt **источник** - это группа **ресурсов** (например, все ваши данные из Hubspot)
* dlt **ресурс** - это функция, которая выдает данные (например, функция, которая выдает все ваши компании из Hubspot)
* dlt **конвейер** - это то, как вы загружаете данные.

Загрузка данных состоит из нескольких шагов:
1. Используйте библиотеку запросов для получения данных
2. Определите dlt-ресурс, который выдает отдельные документы
3. Создайте конвейер dlt и запустите его

In [ ]:
import requests
import dlt

qa_dataset = requests.get("https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1").json()

@dlt.resource
def qa_documents():
  for course in qa_dataset:
    yield course["documents"]

pipeline = dlt.pipeline(pipeline_name="from_json", destination="lancedb", dataset_name="qanda")

load_info = pipeline.run(qa_documents, table_name="documents")

print(load_info)

documents
[{'name': 'text', 'data_type': 'text', 'nullable': True}, {'name': 'section', 'data_type': 'text', 'nullable': True}, {'name': 'question', 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
_dlt_loads
[{'name': 'load_id', 'data_type': 'text', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': True}, {'name': 'status', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_version_hash', 'data_type': 'text', 'nullable': True}]
_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type': 'text', 'nullable': False}, {'name': 'created_at', 'data_type': 'timestamp'

In [ ]:
import lancedb

db = lancedb.connect("./.lancedb")
print(db.table_names())

['qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents']


In [ ]:
db_table = db.open_table("qanda___documents")

db_table.to_pandas()

,id__,text,section,question,_dlt_load_id,_dlt_id
0,e459be09-624d-5f42-a73e-f08d0fe46437,The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1720450988.469787,s+AobaLQwjaVVw
1,21bf11a6-1fd0-5651-b787-b5d32f0c1276,GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1720450988.469787,8shGeLQK2Ol0Ug
2,70416f95-5d54-5d0c-9b38-71b8a843bb76,"Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1720450988.469787,iA1N56McTgL4bw
3,85e03bfb-273c-5de7-b0ae-a347701dcb09,You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1720450988.469787,sVnCjk/WOs0C6w
4,cd6dcd45-7cb0-5d7f-b9fa-6b6875a823cc,You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1720450988.469787,NOyyBTSnyZgjLw
...,...,...,...,...,...,...
943,55959721-31fc-5403-8b4b-109d7b4dacdc,Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1720450988.469787,g90Nxts+Vt+Q+w
944,c922290f-d042-5247-b442-75cb15c88e3f,Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1720450988.469787,dUd9limXi1NcHA
945,7735d566-330c-5678-b2be-4e2210b2960b,Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1720450988.469787,AmpkXvHGg2eLzg
946,5ef7e3c3-bb0f-5ce0-8fed-18b5460ef4ee,Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1720450988.469787,YJxZVY7rhCuETg


## Load and embed the data

Теперь мы снова загрузим те же данные (в новую таблицу), но внедрим их непосредственно с помощью `lancedb_adapter`. Это состоит из следующих шагов:


1. Определите модель встраивания с помощью переменных ENV.
2. определите новый конвейер для загрузки тех же данных и встраивания столбцов "текст" и "вопрос" с помощью `lancedb_adapter`.


Вы можете использовать любую модель встраивания, от open source до OpenAI. Мы выбрали трансформатор предложений [`all-MiniLM-L6-v2`](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) для скорости и простоты.


Примечание: этот конвейер работает немного дольше, поскольку ему приходится загружать модель и встраивать данные.

In [ ]:
import os
from dlt.destinations.adapters import lancedb_adapter

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

pipeline = dlt.pipeline(pipeline_name="from_json_embedded", destination="lancedb", dataset_name="qanda_embedded")

load_info = pipeline.run(lancedb_adapter(qa_documents, embed=["text", "question"]), table_name="documents")
print(load_info)

documents
[{'name': 'text', 'x-lancedb-embed': True, 'data_type': 'text', 'nullable': True}, {'name': 'section', 'data_type': 'text', 'nullable': True}, {'name': 'question', 'x-lancedb-embed': True, 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

_dlt_loads
[{'name': 'load_id', 'data_type': 'text', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': True}, {'name': 'status', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_version_hash', 'data_type': 'text', 'nullable': True}]
_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type': 'text', 'nullable': False}, {'name': 'created_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
_dlt_version
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version'

In [ ]:
db = lancedb.connect("./.lancedb")
print(db.table_names())

['qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents', 'qanda_embedded____dlt_loads', 'qanda_embedded____dlt_pipeline_state', 'qanda_embedded____dlt_version', 'qanda_embedded___dltSentinelTable', 'qanda_embedded___documents']


In [ ]:
db_table = db.open_table("qanda_embedded___documents")

db_table.to_pandas()

,id__,vector__,text,section,question,_dlt_load_id,_dlt_id
0,e9352538-6d00-5241-acf4-4c9eec53b1f9,"[-0.00035095983, -0.062014256, -0.03799991, 0....",The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1720451176.8967116,czzr5C4btuRVng
1,33e203c0-ff17-598b-9b7f-5d1f859862b8,"[0.020011423, -0.011535534, 0.0130172055, -0.0...",GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1720451176.8967116,y5vcyyN8XF1FwA
2,6aadca6c-7b3b-58d6-95a3-c3ee3a0d554c,"[0.014857549, -0.06664994, -0.013571203, 0.023...","Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1720451176.8967116,eYbHFDG9+zJYqA
3,dfaf283a-4eec-5910-b6af-6e8358b0da9f,"[-0.023312101, -0.09461491, 0.056361604, -0.00...",You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1720451176.8967116,S5URdqjpH0erFw
4,dc6f7e10-f666-5936-a606-6da3693ecd12,"[0.026537659, -0.017796658, 0.0021156375, 0.00...",You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1720451176.8967116,7UhL4Ht1vq9efg
...,...,...,...,...,...,...,...
943,e44747c1-420c-5667-a297-a372b8d08087,"[0.016619304, -0.033603117, -0.09334721, -0.02...",Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1720451176.8967116,xonIGXx2qrr9DA
944,8668df2d-1390-5819-b099-e840b9371cb7,"[0.026872871, -0.0019949335, 0.008369085, -0.0...",Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1720451176.8967116,PzMP/5BIos6jqA
945,1b67e2af-b14c-5fef-b862-d02f0e671355,"[0.03513752, 0.056265578, 0.024428517, -0.0651...",Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1720451176.8967116,LCtIQsl0M/H8Bg
946,8c8899df-8a25-5e26-8f4d-a0d8cf112b93,"[0.03380982, -0.0031219546, 0.0017484119, 0.01...",Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1720451176.8967116,EnYnSL5iYaOenw


Вот и все для этого вводного примера! Теперь БД можно использовать в качестве основы для RAG.

# Create an up-to-date RAG with dlt and LanceDB

В этой демонстрации мы создадим чат-бота LLM, который будет обладать последними знаниями о справочнике сотрудников вымышленной компании. Мы сможем общаться с ним по поводу конкретных правил, таких как PTO, работа из дома и т. д.

Чтобы создать такую систему, нам нужно сделать три вещи:
1. Политика компании существует в [Notion Page](https://dlthub.notion.site/Employee-handbook-669c2a1e04044465811c8ca22977685d). Сначала нам нужно извлечь текст из этих страниц.
2. После извлечения мы захотим встроить их в векторы, а затем сохранить в векторной базе данных.
3. Это позволит нам создать RAG: функцию, которая будет принимать вопрос пользователя, сопоставлять его с информацией, хранящейся в векторной базе данных, а затем отправлять вопрос + соответствующую информацию в качестве входных данных в LLM.

Для этого мы будем использовать следующие инструменты OSS:
1. dlt для ввода данных:
1. dlt может легко подключаться к любому источнику REST API (например, Notion).
2. у него также есть интеграция с векторными базами данных, например LanceDB.
3. он также позволяет легко подключать функциональность, например инкрементную загрузку.
2. LanceDB как векторная база данных:
1. LanceDB - это векторная база данных с открытым исходным кодом, которую очень легко использовать и интегрировать в рабочие процессы на python.
2. она работает в процессе и без сервера (как DuckDB), что делает запросы и восстановление очень эффективными
3. Ollama для RAG:
1. Ollama имеет открытый исходный код и позволяет легко запускать LLM локально

**Примечаниепо запуску этого блокнота**: Мы собираемся загрузить и использовать локальный экземпляр Ollama для RAG, поэтому при запуске этого блокнота желательно выбрать **T4 GPU** в среде выполнения (Runtime > Change runtime type > Hardware accelerator > T4 GPU).

Вы также можете использовать процессор по умолчанию, если у вас возникли технические проблемы, но тогда ваши ответы LLM могут быть медленнее (~2 минуты на ответ)

## Part 1: Create a Notion -> LanceDB pipeline using dlt

### 1. Install requirements

Чтобы создать конвейер notion -> lancedb, нам нужно установить:
1. dlt с дополнительными возможностями lancedb
2. sentence-transformers: нам нужно использовать модель встраивания для векторизации и хранения данных в LanceDB. Для этого мы выбираем модель с открытым исходным кодом "sentence-transformers/all-MiniLM-L6-v2".

In [ ]:
%%capture
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers

### 2. Create a dlt project with rest_api source and lancedb destination

Теперь мы создадим проект dlt с помощью команды `dlt init <source><destination>`.

Это загрузит все модули, необходимые для источника dlt (в данном случае rest api), в локальную директорию. Созданную структуру каталогов смотрите на боковой панели.

Что такое источник dlt rest api?

Это источник dlt, который позволяет вам подключаться к любой конечной точке REST API, используя декларативную конфигурацию. Вы можете:
- передать конечные точки, к которым вы хотите подключиться,
- определить связь между конечными точками
- определите, как вы хотите обрабатывать пагинацию и аутентификацию

In [ ]:
!yes | dlt init rest_api lancedb

Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
Cloning and configuring a verified source rest_api (Generic API Source)
Do you want to proceed? [Y/n]: 
Verified source rest_api was added to your project!
* See the usage examples and code snippets to copy from rest_api_pipeline.py
* Add credentials for lancedb and other secrets in ./.dlt/secrets.toml
* requirements.txt was created. Install it with:
pip3 install -r requirements.txt
* Read https://dlthub.com/docs/walkthroughs/create-a-pipeline for more information


### 3. Add API credentials

Чтобы получить доступ к API, базам данных или любым сторонним приложениям, может потребоваться указать соответствующие учетные данные.

С помощью dlt мы можем сделать это двумя способами:
1. Передать учетные данные и любую другую конфиденциальную информацию внутри `.dlt/secrets.toml`.
  ```toml
  [sources.rest_api.notion]
  api_key = "notion api key"

  [destination.lancedb]
  embedding_model_provider = "sentence-transformers"
  embedding_model = "all-MiniLM-L6-v2"

  [destination.lancedb.credentials]
  uri = ".lancedb"
  api_key = "api_key"
  embedding_model_provider_api_key = "embedding_model_provider_api_key"
  ```
2. Pass them as environment variables
  ```python
  import os
  
  os.environ["SOURCES__REST_API__NOTION__API_KEY"] = "notion api key"

  os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
  os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

  os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"
  os.environ["DESTINATION__LANCEDB__CREDENTIALS__API_KEY"] = "api_key"
  os.environ["DESTINATION__LANCEDB__CREDENTIALS__EMBEDDING_MODEL_PROVIDER_API_KEY"] = "embedding_model_provider_api_key"
  ```

Мы будем использовать вариант 2. Не рекомендуется вставлять конфиденциальную информацию вроде API-ключей в код, поэтому вместо этого мы включим их во вкладку секретов в боковой панели блокнота. Это позволит нам получить доступ к секретным значениям из блокнота.

Поскольку мы используем OSS-версию LanceDB и OSS-модели встраивания, нам нужно указать только API-ключ для Notion.

**Примечание**: Вам нужно будет скопировать [ключ API Notion](https://share.1password.com/s#ohRHKjRIGagH_7HzxHzieZViCefOUmodTs2vodixXdQ ) во вкладку секретов под именем `SOURCES__REST_API__NOTION__API_KEY`. После вставки ключа не забудьте включить доступ к ноутбуку.

In [ ]:
import os
from google.colab import userdata

os.environ["SOURCES__REST_API__NOTION__API_KEY"] = userdata.get("SOURCES__REST_API__NOTION__API_KEY")

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

### 4. Write the pipeline code

**Примечание**: Сначала мы пройдемся по коду шаг за шагом, прежде чем поместить его в запускаемые ячейки

1. Импортируйте необходимые модули (запустите эту ячейку)

In [ ]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

2. Настройте источник dlt rest api для подключения и извлечения соответствующих данных из Notion REST API.

Наше пространство понятий состоит из нескольких страниц, и на каждой странице есть несколько параграфов (называемых блоками). Чтобы извлечь все эти данные из Notion API, нам сначала нужно получить список всех page_id (каждая страница имеет уникальный page_id), а затем использовать page_id для запроса содержимого отдельных страниц. А именно:
1. сначала мы запросим идентификаторы страниц из конечной точки `/search`.
2. затем, используя возвращенные идентификаторы страниц, мы запросим содержимое из конечной точки `/blocks/{page_id}/children`.

Исходя из этого, мы можем настроить источник dlt notion rest api следующим образом:
  ```python
  RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "workshop",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }
    ```
Пояснения:
1. `client`: Здесь мы добавили наш базовый url, заголовки и аутентификацию.
2. `resources`: Это список конечных точек, с которых мы хотим запросить данные (здесь: `/search` и `/blocks/{page_id}/children`)
3. [`/search`](https://developers.notion.com/reference/post-search) конечная точка:
- Конечная точка поиска Notion API позволяет нам фильтровать страницы по названию. Мы можем указать, какие страницы мы хотим вернуть, основываясь на параметре "query". Например, если мы хотим вернуть только те страницы, в заголовке которых есть слово "workshop", то мы зададим `"query": "workshop"` в теле json.
- В качестве ответа он возвращает только метаданные страницы (например, page_id). Пример ответа:
      ```json
          {
            "object": "list",
            "results": [
              {
                "object": "page",
                "id": "954b67f9-3f87-41db-8874-23b92bbd31ee",
                "created_time": "2022-07-06T19:30:00.000Z",
                "last_edited_time": "2022-07-06T19:30:00.000Z",
                .
                .
                .
            ],
            "next_cursor": null,
            "has_more": false,
            "type": "page_or_database",
            "page_or_database": {}
          }
      ```
      -Вот как мы определим конфигурацию конечной точки для `/search`:
      ```python
           {
             "name": "search",
             "endpoint": {
                 "path": "search",
                 "method": "POST",
                 "paginator": PostBodyPaginator(),
                 "json": {
                     "query": "workshop",
                     "sort": {
                         "direction": "ascending",
                         "timestamp": "last_edited_time"
                     }
                 },
                 "data_selector": "results"
             }
         },
      ```
- `paginator` позволяет нам указать стратегию пагинации, соответствующую API и конечной точке. (Подробнее об этом позже)
- Поскольку `/search` является конечной точкой POST, мы можем включить json-тело в ключ `json`.
- Нам не нужен весь JSON-ответ, а только содержимое поля "results". Мы отфильтруем его, указав `"data_selector": "results"`.

4. Конечная точка [`blocks/{page_id}/children`](https://developers.notion.com/reference/get-block-children):
- Это точка GET, которая возвращает список объектов блоков (в нашем случае параграфов) с определенной страницы.
- Поскольку он принимает page_id в качестве параметра, мы можем передать его внутри ключа `params`.
- Мы хотели бы иметь возможность автоматически получать идентификаторы страниц, возвращаемые из конечной точки `/search`, и передавать их в качестве параметра в конечную точку `blocks/{page_id}/children`. Мы можем сделать это, связав два ресурса следующим образом:

      ```python
      {
            "name": "page_content",
            "endpoint": {
                "path": "blocks/{page_id}/children",
                "paginator": JSONResponsePaginator(),
                "params": {
                    "page_id": {
                        "type": "resolve",
                        "resource": "search",
                        "field": "id"
                    }
                },
            }
      }
      ```
      - Указывая `"type": "resolve"`, мы сообщаем dlt, что этот параметр должен быть разрешен из родительского ресурса `"search"` с помощью поля `"id"`, которое соответствует id страницы в ответе `/search`.

Примечание по поводу пагинации:

Различные REST API могут использовать различные стратегии для обработки пагинации ответов. dlt имеет встроенную поддержку [наиболее распространенных механизмов пагинации](https://dlthub.com/docs/general-usage/http/rest-client#paginators), и они могут быть явно переданы в конфигурации, как показано выше.

Однако в большинстве случаев нет необходимости явно указывать стратегию пагинации, так как dlt определяет ее автоматически.

В случае если конкретная пагинация пока не поддерживается dlt, вы также можете реализовать собственный пагинатор. Например, в dlt нет встроенного пагинатора для методов POST, поэтому мы пишем свой собственный пагинатор. Мы берем [код, представленный в документации по нему](https://dlthub.com/docs/general-usage/http/rest-client#example-2-creating-a-paginator-for-post-requests), и вносим в него небольшие изменения, основываясь на [документации по понятным API](https://developers.notion.com/reference/intro#parameters-for-paginated-requests).

  ```python
  class PostBodyPaginator(BasePaginator):
      def __init__(self):
          super().__init__()
          self.cursor = None

      def update_state(self, response: Response) -> None:
          # Assuming the API returns an empty list when no more data is available
          if not response.json():
              self._has_next_page = False
          else:
              self.cursor = response.json().get("next_cursor")
              if self.cursor is None:
                  self._has_next_page = False

      def update_request(self, request: Request) -> None:
          if request.json is None:
              request.json = {}

          # Add the cursor to the request body
          request.json["start_cursor"] = self.cursor
  ```

3. Извлечение релевантного содержимого из тела ответа


Ответ, возвращаемый API, представляет собой вложенный JSON, который нам нужно предварительно обработать, прежде чем использовать его где-либо. dlt может автоматически разложить json, но поскольку API Notion немного запутанный, лучше сначала предварительно обработать его, чтобы в результате получить более структурированную БД.
Один из способов сделать это - передать JSON-ответ через функцию преобразования, которая извлечет из JSON-тела только релевантные данные (позже мы добавим их как отображение к ресурсу):

  ```python
  def extract_page_content(response):
      block_id = response["id"]
      last_edited_time = response["last_edited_time"]
      block_type = response.get("type", "Not paragraph")
      if block_type != "paragraph":
          content = ""
      else:
          try:
              content = response["paragraph"]["rich_text"][0]["plain_text"]
          except IndexError:
              content = ""
      return {
          "block_id": block_id,
          "block_type": block_type,
          "content": content,
          "last_edited_time": last_edited_time,
          "inserted_at_time": datetime.now(timezone.utc)
      }
  ```
Здесь также можно применить какую-либо стратегию разбивки на абзацы, но в данном примере мы опустим этот момент, поскольку текст Notion уже предварительно разбит на абзацы. Любая предварительная обработка данных также может происходить здесь.


**Примечание**: Если вы хотите включить родительскую страницу в возвращаемые данные, вы можете сделать это, включив `response["parent"]["page_id"]`. Смотрите пример ответа 200 в [Notion docs](https://developers.notion.com/reference/get-block-children).
JSON-ответ перед функцией:
  ```
  {
      "object": "list",
      "results": [
        {
          "object": "block",
          "id": "c02fc1d3-db8b-45c5-a222-27595b15aea7",
          "created_time": "2022-03-01T19:05:00.000Z",
          "last_edited_time": "2022-03-01T19:05:00.000Z",
          .
          .
          .
          "type": "paragraph",
          "paragraph": {
            "rich_text": [
              {
                .
                .
                .
                "annotations": {
                  .
                  .
                  .

                },
                "plain_text": "Lacinato kale is a variety of kale with a long tradition in Italian cuisine, especially that of Tuscany. It is also known as Tuscan kale, Italian kale, dinosaur kale, kale, flat back kale, palm tree kale, or black Tuscan palm.",
                "href": "https://en.wikipedia.org/wiki/Lacinato_kale"
              }
            ],
            "color": "default"
          }
        }
      ],
      "next_cursor": null,
      "has_more": false,
      "type": "block",
      "block": {}
  }
  ```
 После прохождения через функцию преобразования:

  ```
  {
      "block_id": "c02fc1d3-db8b-45c5-a222-27595b15aea7",
      "block_type": "paragraph",
      "content": "Lacinato kale is a variety of kale with a long tradition in Italian cuisine, especially that of Tuscany. It is also known as Tuscan kale, Italian kale, dinosaur kale, kale, flat back kale, palm tree kale, or black Tuscan palm.",
      "last_edited_time": "2022-03-01T19:05:00.000Z",
  }

    ```

4. инкрементная загрузка данных

Инкрементная загрузка - очень важный аспект построения масштабируемых конвейеров данных. Это техника загрузки только новых или измененных данных с момента последнего запуска конвейера.

В нашем случае при первом запуске конвейера все параграфы из справочника сотрудников будут загружены в виде отдельных строк в таблицу lancedb. Теперь, если мы изменим содержимое одного из параграфов и повторно запустим конвейер для обновления таблицы, то без выполнения инкрементной загрузки может произойти одно из двух, в зависимости от выбранного нами варианта:
- Если мы выберем опцию "replace", то существующие данные в lancedb будут удалены, а все параграфы будут загружены заново.
- Если мы выберем опцию "append", то существующие строки останутся, а все параграфы будут загружены заново в виде новых строк, что приведет к увеличению количества строк в два раза.

Чтобы гарантировать, что загружаются только новые/измененные строки, нам понадобятся следующие элементы:
- Колонка, которая может отслеживать изменения в строке (Пример: загружать только строки, в которых `last_edited_time` больше, чем текущее максимальное `last_edited_time`)
- Столбец primary_key, который однозначно идентифицирует строку, чтобы можно было отследить, когда она изменилась
- Стратегия разрешения изменений в одном ряду (пример: отбросить текущий и загрузить измененный ряд).


Это поведение можно легко настроить в ресурсе dlt:
- Передайте инкрементный столбец в качестве параметра внутри ресурса
    ```python
    def rest_api_notion_incremental(
      last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
    ):
    ```
Мы выбрали столбец `last_edited_time`, поскольку он отслеживает, когда изменяется абзац.
- Передайте следующие аргументы в `@dlt.resource`, чтобы определить стратегию работы с дублирующимися строками:
- `write_disposition="merge"`: гарантирует, что все дублирующиеся строки будут объединены по первичному ключу
- `primary_key="block_id"`: указывает первичный ключ, по которому мы хотим объединить строки. В нашем случае это `block_id`, который является уникальным идентификатором, соответствующим каждому блоку (абзацу).
- `columns={"last_edited_time":{"dedup_sort": "desc"}}`: здесь задается стратегия дедупликации (как мы хотим разрешить дублирование строк). Здесь мы решили оставить строку с наибольшим значением `last_edited_time`.


Собираем все вместе:

    ```python
    @dlt.resource(
        name="employee_handbook",
        write_disposition="merge",
        primary_key="block_id",
        columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
    def rest_api_notion_incremental(
        last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
    ):
        for block in rest_api_notion_resource.add_map(extract_page_content):   
            if not(len(block["content"])):
                continue
            yield block
  ```
Здесь `rest_api_notion_resoure` дает JSON-ответ от Notion REST API, а `extract_page_content` - это функция преобразования, через которую мы передаем JSON-ответ..

5. Создайте конвейер и запустите его


Теперь, когда наш источник настроен, мы можем определить конвейер и запустить его.


Обычно для этого мы запускаем
  ```python
  pipeline.run(
    rest_api_notion_incremental,
    table_name="employee_handbook",
    write_disposition="merge"
  )
  ```
и это загрузит данные в lancedb в обычном режиме, без создания вкраплений.


Однако мы можем заставить lancedb автоматически создавать вкрапления и загружать их вместе с обычными данными, используя собственный адаптер dlt для lancedb: `lancedb_adapter`. Он будет использовать модель встраивания, которую мы указали в учетных данных.
    
  ```python
  pipeline.run(
    lancedb_adapter(
      rest_api_notion_incremental,
      embed="content" # The column that we'd like to embed
    )
    table_name="employee_handbook",
    write_disposition="merge"
  )
  ```

### 5. Run the pipeline

Run this block:

In [ ]:
from datetime import datetime, timezone

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="employee_handbook")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "workshop",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config,name="employee_handbook")

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="employee_handbook",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    # last_value = last_edited_time.last_value
    # print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block

def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="employee_handbook",
        write_disposition="merge"
    )
    print(load_info)

load_notion()

Pipeline company_policies load step completed in 0.24 seconds
1 load package(s) were loaded to destination LanceDB and into dataset notion_pages
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x7fe0d101bcd0> location to store data
Load package 1720452954.1820357 is LOADED and contains no failed jobs


### 6. Visualize the output

In [ ]:
import lancedb

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___employee_handbook")

dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,6adeb540-d180-5d40-bc84-c40e5c173ea1,"[-0.03892389, 0.1208173, 0.046208583, -0.00543...",baac0ba4-9b60-450e-8cc1-1e6e2a0fb7d9,paragraph,"In this section, we describe what we offer to ...",2024-07-03 17:34:00+00:00,2024-07-08 15:30:04.270715+00:00,1720452602.3108296,+LXDpddrXOJUXg
1,cffdb1bb-a146-5e90-8fbb-a1d577a2a98e,"[-0.0799329, 0.13477285, 0.0053403154, -0.0298...",0e429073-6383-4918-8961-fcc66346067f,paragraph,Employee health is important to us. We don’t d...,2024-06-26 08:46:00+00:00,2024-07-08 15:30:04.272891+00:00,1720452602.3108296,gDimzresa+mpsg
2,25cd721d-fd64-517f-9b3b-34e3fad3522e,"[-0.109743185, 0.10586075, 0.003290699, -0.021...",f4e006d7-9b38-49e9-94cf-552beaa75773,paragraph,Our company is dedicated to maintaining a safe...,2024-07-03 17:26:00+00:00,2024-07-08 15:30:04.273102+00:00,1720452602.3108296,1OZTtNPR9Ab8uA
3,c75b7ef9-96b6-551b-9cdd-795bbe01bb6e,"[0.050755523, -0.06461991, 0.06527383, 0.01465...",71618ca5-6c62-4b66-bc0f-3d855e0c4b8b,paragraph,If your job doesn’t require you to be present ...,2024-06-26 08:52:00+00:00,2024-07-08 15:30:04.273273+00:00,1720452602.3108296,t3k2vgTDh3Fc7Q
4,7a69c4c0-cd55-5090-903e-facf23eadde5,"[0.00052337867, -0.054883413, 0.043573413, -0....",cd15aaf5-6cdc-4a13-835c-2181fd7bf81e,paragraph,Remote working refers to working from a non-of...,2024-07-03 17:19:00+00:00,2024-07-08 15:30:04.273443+00:00,1720452602.3108296,R5wmdAkkOOmdpg
5,ff1141dc-88f6-500a-a8c3-c18e37661650,"[0.03802633, -0.021509705, 0.04752782, 0.06470...",a4b2f0c9-e0c8-4b3c-81e7-ef624809977d,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:32:00+00:00,2024-07-08 15:30:04.273595+00:00,1720452602.3108296,qxJLuSZQaZq/fw
6,71e89a85-ae0b-5b68-866b-bd3922ec7548,"[-0.055131722, -0.07363651, 0.032283936, 0.009...",c0262981-b5f1-4a57-a91f-2e75f649b86c,paragraph,[edited] Our company operates between 9 a.m. t...,2024-07-08 13:06:00+00:00,2024-07-08 15:30:04.662363+00:00,1720452602.3108296,IHR3Vw8v8tyJZQ
7,a28e913f-761f-5684-8cd5-0d0c49e0338c,"[-0.004968941, -0.003911972, 0.028705625, 0.00...",faacf4ec-90be-4e96-b8b9-29b5112bc7ca,paragraph,Employees receive [20 days] of Paid Time Off (...,2024-06-26 09:03:00+00:00,2024-07-08 15:30:04.662656+00:00,1720452602.3108296,/oDmr/7ulovYhQ
8,a18932d9-1583-5c42-bd0d-0f96738c5e6c,"[0.032060888, 0.024244698, 0.008471344, 0.0317...",e6021a51-f403-4950-80c2-ebff005c7289,paragraph,Our company observes the following holidays: N...,2024-06-26 09:08:00+00:00,2024-07-08 15:30:04.662820+00:00,1720452602.3108296,9I7CX2AaReDvng
9,93661874-13a2-5a43-bed8-868005dfd5e2,"[-0.0131553095, 0.008382407, 0.017044391, 0.05...",b8f4cc6d-c28c-4071-9545-caadce5eb37b,paragraph,These holidays are considered “off-days” for m...,2024-06-26 09:09:00+00:00,2024-07-08 15:30:04.662974+00:00,1720452602.3108296,HiI2XYEzmAEQMA


 ---

Теперь мы вносим изменения в один из параграфов и снова запускаем конвейер, чтобы увидеть эффект от инкрементной загрузки. Мы наблюдаем две вещи:
1. столбец `inserted_at_time` изменился только для обновленного ряда, что означает, что только этот ряд был добавлен
2. посмотрев на первичный ключ `block_id`, мы видим, что исходный ряд был удален, а обновленный - вставлен

In [ ]:
db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___employee_handbook")

dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,6adeb540-d180-5d40-bc84-c40e5c173ea1,"[-0.03892389, 0.1208173, 0.046208583, -0.00543...",baac0ba4-9b60-450e-8cc1-1e6e2a0fb7d9,paragraph,"In this section, we describe what we offer to ...",2024-07-03 17:34:00+00:00,2024-07-08 15:30:04.270715+00:00,1720452602.3108296,+LXDpddrXOJUXg
1,cffdb1bb-a146-5e90-8fbb-a1d577a2a98e,"[-0.0799329, 0.13477285, 0.0053403154, -0.0298...",0e429073-6383-4918-8961-fcc66346067f,paragraph,Employee health is important to us. We don’t d...,2024-06-26 08:46:00+00:00,2024-07-08 15:30:04.272891+00:00,1720452602.3108296,gDimzresa+mpsg
2,25cd721d-fd64-517f-9b3b-34e3fad3522e,"[-0.109743185, 0.10586075, 0.003290699, -0.021...",f4e006d7-9b38-49e9-94cf-552beaa75773,paragraph,Our company is dedicated to maintaining a safe...,2024-07-03 17:26:00+00:00,2024-07-08 15:30:04.273102+00:00,1720452602.3108296,1OZTtNPR9Ab8uA
3,c75b7ef9-96b6-551b-9cdd-795bbe01bb6e,"[0.050755523, -0.06461991, 0.06527383, 0.01465...",71618ca5-6c62-4b66-bc0f-3d855e0c4b8b,paragraph,If your job doesn’t require you to be present ...,2024-06-26 08:52:00+00:00,2024-07-08 15:30:04.273273+00:00,1720452602.3108296,t3k2vgTDh3Fc7Q
4,7a69c4c0-cd55-5090-903e-facf23eadde5,"[0.00052337867, -0.054883413, 0.043573413, -0....",cd15aaf5-6cdc-4a13-835c-2181fd7bf81e,paragraph,Remote working refers to working from a non-of...,2024-07-03 17:19:00+00:00,2024-07-08 15:30:04.273443+00:00,1720452602.3108296,R5wmdAkkOOmdpg
5,ff1141dc-88f6-500a-a8c3-c18e37661650,"[0.03802633, -0.021509705, 0.04752782, 0.06470...",a4b2f0c9-e0c8-4b3c-81e7-ef624809977d,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:32:00+00:00,2024-07-08 15:30:04.273595+00:00,1720452602.3108296,qxJLuSZQaZq/fw
6,a28e913f-761f-5684-8cd5-0d0c49e0338c,"[-0.004968941, -0.003911972, 0.028705625, 0.00...",faacf4ec-90be-4e96-b8b9-29b5112bc7ca,paragraph,Employees receive [20 days] of Paid Time Off (...,2024-06-26 09:03:00+00:00,2024-07-08 15:30:04.662656+00:00,1720452602.3108296,/oDmr/7ulovYhQ
7,a18932d9-1583-5c42-bd0d-0f96738c5e6c,"[0.032060888, 0.024244698, 0.008471344, 0.0317...",e6021a51-f403-4950-80c2-ebff005c7289,paragraph,Our company observes the following holidays: N...,2024-06-26 09:08:00+00:00,2024-07-08 15:30:04.662820+00:00,1720452602.3108296,9I7CX2AaReDvng
8,93661874-13a2-5a43-bed8-868005dfd5e2,"[-0.0131553095, 0.008382407, 0.017044391, 0.05...",b8f4cc6d-c28c-4071-9545-caadce5eb37b,paragraph,These holidays are considered “off-days” for m...,2024-06-26 09:09:00+00:00,2024-07-08 15:30:04.662974+00:00,1720452602.3108296,HiI2XYEzmAEQMA
9,b220778f-1118-5c22-b614-3bc0fd0a602b,"[0.027987516, 0.067343615, 0.03980646, 0.00774...",ea7a1beb-6874-4f41-966d-dc1f80a1f635,paragraph,Employees who are unable to work due to illnes...,2024-06-26 09:11:00+00:00,2024-07-08 15:30:04.663125+00:00,1720452602.3108296,UwkDg5Htn2kTvA


## Part 2: Create a RAG bot using Ollama

Теперь, когда содержимое справочника сотрудника векторизовано и сохранено в LanceDB, мы готовы к созданию RAG с помощью Ollama.

Что такое RAG?

Автоматизированная генерация поиска (RAG) - это система извлечения релевантных документов из базы данных и передачи их вместе с запросом в LLM, чтобы LLM мог генерировать ответы с учетом контекста.

В нашем случае, если бы мы задали LLM вопросы о политике наших сотрудников, мы бы не получили полезных ответов, потому что LLM никогда не видел этих политик. Решением этой проблемы может стать вставка всех политик в подсказку, а затем задавать вопросы. Однако это не представляется возможным, учитывая ограничения на размер контекстного окна.

Мы можем обойти это ограничение с помощью RAG:
1. задав вопрос пользователя, мы сначала вложим его в вектор
2. затем мы выполним векторный поиск в нашей таблице LanceDB и получим k лучших результатов - наиболее релевантные абзацы, соответствующие вопросу
3. наконец, мы передаем исходный вопрос вместе с полученными параграфами в качестве подсказки в LLM


1. Install Ollama into the notebook's local runtime

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


2. Start Ollama using `ollama serve`. This needs to run in the backgound - so we run it using `nohup` (to see the output log, open nohup.out).

In [ ]:
!nohup ollama serve > nohup.out 2>&1 &

3. Pull the desired model. We're going to be using `llama1-uncensored` (takes about 1m to download)

In [ ]:
%%capture
!ollama pull llama2-uncensored

In this next part we're going to be writing functions that accept user question, retrieve the relevant paragraphs from lancedb, and the pass the question and the retrieved pages as input into the ollama chat assistant

4. pip install ollama and import it

In [ ]:
!pip install ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.2 MB/s eta 0:00:00


In [ ]:
import ollama

5. напишите функцию, которая может получить из lancedb содержимое, соответствующее запросу пользователя
С LanceDB вам не нужно явно встраивать вопрос. LanceDB хранит информацию об используемой модели встраивания и автоматически встраивает вопрос.

Мы используем функцию `db_table.search()` для запроса к БД, а затем ограничиваем ее двумя наиболее похожими результатами и возвращаем их в качестве контекста для передачи в RAG.

Ограничение результатов важно, потому что в противном случае может быть слишком много путаной информации. Аналогичным образом, выбор только самого лучшего варианта может не дать достаточной информации.

In [ ]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

6. Наконец, мы определяем очень простой RAG. Мы определяем простую системную подсказку, получаем соответствующий контекст для запроса пользователя с помощью функции, определенной выше, а затем отправляем вопрос пользователя и контекст в модель `llama2-uncensored`.

In [ ]:
def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table("notion_pages___employee_handbook")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        model="llama2-uncensored",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

И мы запускаем RAG! Некоторые примеры вопросов, которые вы можете задать:

* Сколько дней отпуска я получу?
* Могу ли я получить декретный отпуск?

**Примечание**: Это очень базовая реализация RAG, поскольку этот семинар в основном посвящен вводу данных. Поэтому ожидайте странных ответов. Если вы остановите и перезапустите ячейку, вам нужно будет сначала запустить ячейку, содержащую `ollama serve`.

In [ ]:
main()

You: How many vacation days do I get?
Assistant: Based on the provided context, the employee handbook states that employees are entitled to eight (8) paid vacation days per year. The first step would be to ask if they have been employed with the company for at least 90 days or more, as some companies may offer additional PTO upon meeting this requirement. If it has been over 90 days since employment, please provide details on what other options the employee has for PTO if they have not taken their floating day yet.
If the employee is an exempt employee, they will receive an additional day of PTO that they must take within 12 months after any holiday observed by the company. The employee should check with HR to see what holidays are counted for this purpose and if there are any special rules or requirements that apply. If it has been over 12 months since any holiday was observed, then the employee may be able to use their PTO for a personal day instead of taking an extra day off from wo

KeyboardInterrupt: Interrupted by user

Многое еще можно узнать и сделать с помощью dlt и LanceDB, более подробную информацию вы найдете в [dlt docs](https://dlthub.com/docs/ ) и [LanceDB docs](https://lancedb.github.io/lancedb/).

Если у вас есть вопросы по этому семинару или dlt, присоединяйтесь к нашему [сообществу на Slack](https://dlthub.com/community).

Если вы будете на EuroPython в Праге на этой неделе, загляните к нам на стенд!